In [32]:
# Import Splinter and Browser
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

# With the line above we are accomplishing two things.
# 1. We're searching for elements with a specific combination of tag (div) and attribute (list_text).
# 2. We're telling our browser to wait one second before searching for components. The optional delay is 
# useful to allow dynamic pages time to load while to laod, esspecially if they are image-heavy.

True

In [9]:
# Add the HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

### Title of Articles

In [19]:
# Start Scraping
# Use the parent element to find the first 'a' tag and save it as news_title
news_title = slide_elem.find('div', class_='content_title').text
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').text
print(news_title)
print('-')
print(news_p)

NASA's Perseverance Mars Rover Gets Balanced
-
The mission team performed a crucial weight-balancing test on the rover in preparation for this summer's history-making launch to the Red Planet.


### Featured Images 

In [23]:
# Visit URL
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [24]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [26]:
# Parese the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [27]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [28]:
# Use the base URL to create an absolut URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

#### Scraping into a DataFrame
- In the first line we are creating a new DataFrame from the HTML table. The Pandas function read_html() specifically searchs for and returns a list of tables found in the HTML. By specifying an index 0, we are telling Pandas to pull only the first table it encounters, or the first item in the list. Then, it turns the table into a DataFrame.
- In the second line we are assigning clumns to the new DataFrame
- Then by using the .set_index() function, we're turning the description column into the DataFrame's index. implace=True means that the updated index will remain in place, without having to reassign the DataFrame to a new variable

In [44]:
# We want to create a dataframe to store our scraped table into.
df = pd.read_html('https://galaxyfacts-mars.com/')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [45]:
browser.quit()